## Data retrieval and cleaning

Don't forget to delete your `json_clean` if you make any modifications `cleaning.py`!


In [1]:
from ift6758.data.acquisition import NHLGameData

data_path_raw = './../../ift6758/data/json_raw/'
nhl_games_data = NHLGameData(data_path_raw)
for year in range(2016,2021):
    nhl_games_data.fetch_season(year)

Loading from cache file ./../../ift6758/data/json_raw/2016/2016-regular.pkl
Found 1230 regular games for season 2016-2017
Loading from cache file ./../../ift6758/data/json_raw/2016/2016-playoff.pkl
Found 102 playoff games for season 2016-2017
Loading from cache file ./../../ift6758/data/json_raw/2017/2017-regular.pkl
Found 1271 regular games for season 2017-2018
Loading from cache file ./../../ift6758/data/json_raw/2017/2017-playoff.pkl
Found 105 playoff games for season 2017-2018
Loading from cache file ./../../ift6758/data/json_raw/2018/2018-regular.pkl
Found 1271 regular games for season 2018-2019
Loading from cache file ./../../ift6758/data/json_raw/2018/2018-playoff.pkl
Found 105 playoff games for season 2018-2019
Loading from cache file ./../../ift6758/data/json_raw/2019/2019-regular.pkl
Found 1271 regular games for season 2019-2020
Loading from cache file ./../../ift6758/data/json_raw/2019/2019-playoff.pkl
Found 105 playoff games for season 2019-2020
Loading from cache file ./..

In [2]:
from ift6758.data.cleaning import DataCleaner

data_path_clean = './../../ift6758/data/json_clean/'
data_cleaner = DataCleaner(data_raw=nhl_games_data, data_path_clean=data_path_clean)
for year in range(2016,2021):
    data_cleaner.clean_season(year, keepPreviousEventInfo=True, includePowerPlay=True)

In [4]:
import pandas as pd

data_2020 = data_cleaner.get_cleaned_data(2020)

In [5]:
data_2020.head()

,game_id,period,period_time,type,team,x,y,shooter,goalie,shot_type,...,PPActive,PPTimeElapsed,HomeSkaters,AwaySkaters,prev_type,prev_period_time,prev_x,prev_y,time_since_prev,distance_from_prev
0,2020020001,1,00:16,SHOT,Philadelphia Flyers,-74.0,29.0,Travis Konecny,Tristan Jarry,Wrist Shot,...,False,0,5,5,FACEOFF,00:00,0.0,0.0,16,79.48
1,2020020001,1,00:34,SHOT,Pittsburgh Penguins,49.0,-25.0,Evan Rodrigues,Carter Hart,Wrist Shot,...,False,0,5,5,BLOCKED_SHOT,00:28,-58.0,-7.0,6,108.50
2,2020020001,1,01:05,SHOT,Philadelphia Flyers,-52.0,-31.0,Joel Farabee,Tristan Jarry,Backhand,...,False,0,5,5,HIT,01:03,-87.0,-33.0,2,35.06
3,2020020001,1,02:51,SHOT,Pittsburgh Penguins,43.0,39.0,Evan Rodrigues,Carter Hart,Snap Shot,...,False,0,5,5,FACEOFF,02:09,69.0,22.0,42,31.06
4,2020020001,1,03:44,SHOT,Philadelphia Flyers,-53.0,8.0,Claude Giroux,Tristan Jarry,Slap Shot,...,False,0,5,5,GIVEAWAY,03:43,-66.0,4.0,1,13.60


The `NaN` values for previous events is normal as some previous events are not in our interest (the columns are therefore filled with empty values).

In [5]:
data_2020.isna().sum()

game                      0
period                    0
period_time               0
type                      0
team                      0
x                         0
y                         0
shooter                   0
goalie                  277
shot_type                 0
empty_net                 0
strength              50044
opposite_team_side        0
PPActive                  0
PPTimeElapsed             0
HomeSkaters               0
AwaySkaters               0
prev_type                 0
prev_period_time          0
prev_x                 2822
prev_y                 2822
time_since_prev           0
distance_from_prev     2822
dtype: int64

## Feature engineering

In [6]:
from ift6758.features import FeatureEng
data_path_clean = './../../ift6758/data/json_clean/'
w = FeatureEng(data_path_clean)

In [7]:
df = w.features_2(2016,2020)
df.sample(10)

KeyError: 'game_id'

In [ ]:
df.isna().sum()

game_id                       0
period                        0
game_seconds                  0
event_type                    0
x                             0
y                             0
power_play_active             0
power_play_time_elapsed       0
home_skaters                  0
away_skaters                  0
prev_type                     0
prev_x                     4076
prev_y                     4075
time_since_prev               0
distance_from_prev         4076
distance_goal                 0
prev_distance_goal         4076
angle_shot                    0
prev_angle_shot            4076
bounce                        0
angle_change                  2
speed                         0
dtype: int64

Annoying SHOT types that have no coordinates (and therefore no distance or angles)
The second one is a during a shootout. We have to see how we deal with shootout shots that have period times of 0.

In [ ]:
df[df.angle_change.isna()]

,game_id,period,game_seconds,event_type,x,y,power_play_active,power_play_time_elapsed,home_skaters,away_skaters,...,prev_y,time_since_prev,distance_from_prev,distance_goal,prev_distance_goal,angle_shot,prev_angle_shot,bounce,angle_change,speed
48612,2016020801,4,189,GOAL,-83.0,4.0,False,0,5,5,...,NaN,4.0,NaN,8.06,NaN,29.75,NaN,True,NaN,nan
253373,2019020349,5,0,SHOT,-72.0,-1.0,False,0,5,5,...,NaN,0.0,NaN,18.03,NaN,-3.18,NaN,True,NaN,instant


Annoying previous event that only has y coordinate. We keep it for the speed and time between events.

In [10]:
df[df.prev_x.isna() & df.prev_y.notna()]

,game_id,period,game_seconds,event_type,x,y,power_play_active,power_play_time_elapsed,home_skaters,away_skaters,...,prev_y,time_since_prev,distance_from_prev,distance_goal,prev_distance_goal,angle_shot,prev_angle_shot,bounce,angle_change,speed
109854,2017020563,1,910,SHOT,36.0,-29.0,False,0,5,5,...,-17.0,33.0,NaN,61.29,NaN,-28.24,NaN,False,0.0,nan
